# Result Explorer Notebook for xrb Project

## Overview

This notebook allows you to explore the results of xrb model one-zone nucleosynthesis calculations of rapid proton-capture (rp) process  performed by the Model-runner. Results can be organized in different directories (folders) managed by the Model-Runner. 

The results are lists of how much of each isotope was produced or destroyed in the xrb model calculation. 

If things don't work make sure `Python 3` is displayed in the upper right corner of this Tab. If not go there and select it, and then try again. 

## Instructions Step 1: Load Modules
Execute the following cell. This will load the modules for graphing and analysing (we are using the [NuGridPy](https://nugrid.github.io/NuGridPy/) Python package).

In [1]:
%pylab ipympl 
from nugridpy import ppn
from nugridpy import utils
from nugridpy import utils as ut
from nugridpy import ascii_table as at

#### section to include at the beginnning of your notebook 
#### that will suppress output of unnecessary information
import os
from contextlib import contextmanager
@contextmanager
def redirect_stdout(new_target):
    old_target, sys.stdout = sys.stdout, new_target
    try:
        yield new_target
    finally:
        sys.stdout = old_target
def get_devnull():
    #return open(os.devnull, "w")
    return open('log_stuff.txt', "w") #where all the stuff goes you don't want to see
####

Populating the interactive namespace from numpy and matplotlib


## Instructions Step 2: Load Data
Edit the directory (folder) name (the text within `' '` in the first line) and then execute the following cell to load the data. 

You need to specify the directory (folder) that contains the results of the calculation you want to explore. Discuss with your group and the Model-runner what directory (folder) should be chosen (you can see the options on top of the list in the file browser on the left).

You create two Python data instances per case directory. Each of them can make different plots. In the following cell you specify the case directory in the string variable `case_directory` once, and then both instances are created using the same directory name.

In [2]:
case_directory = 'accretion_disk_1' # 'xrb_mesa' 
pa1 = ppn.abu_vector(case_directory)
px1 = ppn.xtime(case_directory)

375 cycle numbers found in accretion_disk_1
Ranging from 00000 to 00374
Range may not be continuous. To display all available cycles, print <abu_vector_instance>.files
number of species:  5214
number of cols:  5220
There are 374 time steps found.


## Instructions Step 3: Visualize Results on the Chart of Nuclides
Executing the following cell creates a chart of nuclides with a color coding that indicates the abundance of each isotope created. Stable isotopes are the thick lined squares. The number indicates the mass number of the isotope. Each row is an element indicated by the element number or proton number Z. This graph helps you identify the most abundant isotopes that have been produced. 

Before you execute the next cell make sure the first argument of `pa1.abu_chart()` is the number of the cycle or time step you want to visualize. For example, if you want to visualize the abundances produced at the end of the calculation - then the number should be the highest cycle number (the range of cycle numbers is displayed as output when you load the data in the cell above). Using an earlier cycle number shows the abundances at an earlier time in the network evolution.

In [5]:
ifig=1;close(ifig);figure(ifig)

plot_cycle = 374

#pa1.abu_chart(plot_cycle,ifig=ifig,ilabel=True,imlabel=True,imlabel_fontsize=5,boxstable=True,\
#              plotaxis=[22.5,60.5,27.5,50.5], lbound=(-10, 0))
pa1.abu_chart(plot_cycle,ifig=ifig,ilabel=True,imlabel=True,imlabel_fontsize=5,boxstable=True,\
              plotaxis=[15,50,15,40], lbound=(-10, 0))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Instructions Step 3: Obtain the abundance number of a specific isotope of interest

As part of your project you will have to write down the exact produced abundance of a specific isotope your group is interested in, for example to determine the factor by which the abundance changed compared to a previous calculation. These are the steps that allow you to do that. 

1. Go to the file explorer on the left and double click on the directory (folder) that contains the results of the calculation you want to analyze (the same directory (folder)) you used for the chart of nuclides plot. 

2. You should see a long list of output files whose names start with `iso_massf...`. The file name also contains a number. Find the file with the highest number - this will contain the abundances at the end of the calculation (a file is created for each step of the calculation). You can make the file browser window wider by sliding the boundary to the right so you can see the full filenames. 

3. Double click on the file with the largest number in the name to open it in a new Tab. 

4. You see a long list of isotopes and their abundances. The last two columns contain element name and mass number to identify the isotope. The number in the column before is the abundance in scientific notation. Scroll down or use the Find feature (`Command - F` on Mac, `CTRL - F` on windows) to find the isotope you are interested in. 

5. Once you are done, click on your group folder in the folder name on top of the file browser (example Group1 or Group2 or Group3) - this should be the name just before the results folder, separated by `/`. This will get you back to the folder (directory) you came from so you can select a different result folder. 

Note the notation of the isotopes is always 5 characters with the first two characters denoting the element symbol in all upper case, and the last three characters the mass number. Element symbol is left justified and mass number is right justified with unused characters being blanks. You need to use that format when searching (type exactly 5 characters). 

## Read in and plot temperature and density as functions of time from `trajectory.input` file used in the xrb model case calculation

In [3]:
# read in trajectory used in the nova model case calculation
traj_name = 'trajectory.input'
traj = at.readTable(filename=case_directory+'/'+traj_name, datatype='trajectory')

#at.readTable?
print ('trajectory head attributes:',traj.hattrs)
print ('trajectory data columns:',traj.dcols)
year_to_minutes = 365.2422*1440
tmin = year_to_minutes*traj.get('time')
t9 = traj.get('T')
rho = traj.get('rho')

trajectory head attributes: {'AGEUNIT': 'SEC', 'TUNIT': 'T9K', 'RHOUNIT': 'CGS'}
trajectory data columns: ['time', 'T', 'rho']


In [4]:
# this is a function that makes a two-axis plot
def two_scales(ax1, time, data1, data2, c1, c2, ls1, ls2, xlbl, ylbl, y2lbl, fsize):
    """

    Parameters
    ----------
    ax1 : axis
        Axis to put two scales on

    time : array-like
        x-axis values for both datasets

    data1: array-like
        Data for left hand scale

    data2 : array-like
        Data for right hand scale

    c1 : color
        Color for line 1

    c2 : color
        Color for line 2        
    
    ls1 : linestyle
        style for line 1

    ls2 : linestyle
        style for line 2
        
    xlbl: string
        xlabel
        
    ylbl: string
        ylabel
        
    y2lbl: string
        label of the 2nd y axis
        
    fsize: font size for axis labels

    Returns
    -------
    ax1 : axis
        Original axis
        
    ax2 : axis
        New twin axis
    """
    ax2 = ax1.twinx()

    ax1.plot(time, data1, color=c1, linestyle=ls1)
    ax1.set_xlabel(xlbl,fontsize=fsize)
    ax1.set_ylabel(ylbl,fontsize=fsize)

    ax2.plot(time, data2, color=c2, linestyle=ls2)
    ax2.set_ylabel(y2lbl,fontsize=fsize)
    return ax1, ax2

# Change color of each axis
def color_y_axis(ax, color):
    """Color your axes."""
    for t in ax.get_yticklabels():
        t.set_color(color)
    return None

In [5]:
# make a plot of trajectory used in the nova model case calculation
fig, ax = subplots()

# select color and style for plotting lines
im1 = 8
a1,b1,c1,d1=utils.linestylecb(im1)
im2 = im1+1
a2,b2,c2,d2=utils.linestylecb(im2)

ax1, ax2 = two_scales(ax, tmin, t9, rho, c1, c2, a1, a2,\
                      'time (min)', '$T_9$',\
                     '$\\rho\ (\mathrm{g\,cm}^{-3})$', 12)
   
color_y_axis(ax1, c1)
color_y_axis(ax2, c2)

ax3 = ax1.twiny()
ax3.set_xticks([])

t9max_cycle = argmax(t9)
title('trajectory for xrb model '+case_directory+', $T_9$ has maximum at cycle '+str(t9max_cycle))

tight_layout()

show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
# for this cell to work xrb case run has to output reaction rate fluxes!!!
# otherwise, skip this cell
# use here the cycle with the maximum T9

flux_cycle = t9max_cycle

flux_file = case_directory+'/flux_'+str(flux_cycle).zfill(5)+'.DAT'
%cp "$flux_file" ./

pa1.flux_solo(flux_cycle,lbound=(-12,0),prange=10,plotaxis=[2.5,37.5,5.5,37.5],profile='neutron',which_flux=0)

tight_layout()
show()

In [12]:
# plot decayed elemental abundances from the xrb model case for plot_cycle
ifig=5;close(ifig);figure(ifig)

sol_ab = 'iniab2.0E-02GN93.ppn' # file with solar abundances

Z_range = [11,71]

with get_devnull() as devnull, redirect_stdout(devnull):
    pa1.elemental_abund(plot_cycle,ref_filename=sol_ab,zrange=Z_range,ylim=[-1,7],\
        label='cycle '+str(plot_cycle), colour='blue',plotlines='--', plotlabels=True, mark='.')

# to compare with other model run uncomment the following two raws
#    pa1a.elemental_abund(plot_cycle,ref_filename=sol_ab,zrange=Z_range,ylim=[-4,5],\
#        label='cycle '+str(plot_cycle), colour='red',plotlines=':', plotlabels=True, mark='.')

grid(False)
xlim(Z_range[0],Z_range[-1])
hlines(0,Z_range[0],Z_range[-1],linestyles='dotted')
ylabel('$\log_{10}\,X_i/X_\odot$')
title('decayed elemental abundances for xrb model '+case_directory+', cycle '+str(plot_cycle))
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Read in solar isotopic abundances

In [10]:
# these are the solar abundances used in nova sims
f = open(sol_ab, 'r')

sol_iso_z=[]
sol_iso=[]
sol_iso_name = []
sol_iso_a = []
sol_iso_abu=[]

for line in f:
    n = len(line.split())
    if n == 3:
        sol_iso = line.split()[1]
        if sol_iso == 'PROT':
            sol_iso_name.append('h')
            sol_iso_a.append(1)
            sol_iso_z.append(int(line.split()[0]))
            sol_iso_abu.append(float(line.split()[2]))
        else:
            sol_iso_name.append(sol_iso[0:2])
            sol_iso_a.append(int(sol_iso[2:5]))
            sol_iso_z.append(int(line.split()[0]))
            sol_iso_abu.append(float(line.split()[2]))
    if n == 4:
        sol_iso_z.append(int(line.split()[0]))
        sol_iso_name.append(line.split()[1])
        sol_iso_a.append(int(line.split()[2]))
        sol_iso_abu.append(float(line.split()[3]))

f.close()

n_iso_sol = len(sol_iso_z)

NameError: name 'sol_ab' is not defined

In [14]:
# read in undecayed isotopic abundances from the xrb model case for the plot_cycle
ppn_file = case_directory+'/'+'iso_massf'+str(plot_cycle).zfill(5)+'.DAT'
print (ppn_file)

f1=open(ppn_file)
lines=f1.readlines()
f1.close()

massfrac=[]
A=[]
Z=[]
element = []
AI = []

for k in range(len(lines)):
    # skip header
    if k<7:
        continue
    
    line=lines[k]
    Z.append(line[6:12].strip()) # Z
    A.append(line[13:17].strip()) # A float
    massfrac.append(line[24:35].strip()) # massf
    element.append(line[37:39].strip()) # element (execept NEUT (first) and PROT (second))
    AI.append(line[39:43].strip()) # A integer

n_iso_ppn = len(A)
element[0] = 'n'
AI[0] = '1'
element[1] = 'H'
AI[1] ='1'
element[n_iso_ppn-2] = 'ALm'
AI[n_iso_ppn-2] ='26'
element[n_iso_ppn-1] = 'KRm'
AI[n_iso_ppn-1] ='85'

iso_z_ppn = np.linspace(0,0,n_iso_ppn)
iso_a_ppn = np.linspace(0,0,n_iso_ppn)
iso_name_ppn = ["  " for x in range(n_iso_ppn)]
    
iso_abu_ppn = np.linspace(0,0,n_iso_ppn)

for i in range(n_iso_ppn):
    iso_name_ppn[i] = element[i]
    iso_a_ppn[i] = float(A[i])
    iso_z_ppn[i] = float(Z[i])
    iso_abu_ppn[i] = float(massfrac[i])

xrb_example/iso_massf01999.DAT


In [18]:
# plot isotopic composition for the xrb model case
ifig=6;close(ifig);fig=figure(ifig)
size=8
fig.canvas.layout.height = str(0.8*size)+'in'   # This is a hack to prevent ipympl
fig.canvas.layout.width  = str(1.2*size)+'in'   # to adjust horizontal figure size

z1 = 1; z2 = 195
for z in range(z1,z2):
    a_plot_ppn = []
    y_plot_ppn = [] 
    for i in range(n_iso_ppn):
        if int(iso_z_ppn[i]) == z:
            for k in range(n_iso_sol):
                if sol_iso_z[k] == z and sol_iso_a[k] == iso_a_ppn[i]:
                    a_plot_ppn.append(sol_iso_a[k])
                    y_plot_ppn.append(log10(iso_abu_ppn[i]/sol_iso_abu[k])) 
    if len(a_plot_ppn) > 0:
        text(a_plot_ppn[argmax(y_plot_ppn)],max(y_plot_ppn),ut.get_el_from_z(z),\
             horizontalalignment='center',verticalalignment='bottom',fontsize=10)
    if len(a_plot_ppn) > 1:
        a_plot_ppn, y_plot_ppn = (list(t) for t in zip(*sorted(zip(a_plot_ppn, y_plot_ppn))))
    plot(a_plot_ppn,y_plot_ppn,'--')
    plot(a_plot_ppn,y_plot_ppn,'bo',markersize=3)

xmin = z1; xmax = z2
hlines(0,xmin,xmax,linestyles='dotted')
xlim(xmin,xmax)
ylim(-3,5.0)
xlabel('mass number')
ylabel('$\log_{10}\,X_i/X_\odot$')
title('undecayed isotopic abundances for xrb model '+case_directory+', cycle '+str(plot_cycle))
show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …